In [21]:
def exerciseBoundary(t, T, K, exBoundType, exBoundParams):
    if len(exBoundParams) == 1:
        if exBoundType == 0:
            B_t = K
        elif exBoundType == 1:
            B_t = K - exBoundParams[0]*np.sqrt(T-t)
        else:
            B_t = K - exBoundType*exBoundParams[0]*np.sqrt(T-t)
    else:
        B_t = K
        if exBoundType == len(exBoundParams):
            for i in exBoundParams:
                B_t -= i * np.sqrt(T-t)
    return max(B_t,0)

In [22]:
def MLMCAmOpLevelL(l, N, NperLoop, T, r, sig, S0, K, M, exBoundType, exBoundParams):
    # l given level --> M^l time steps between 0 and T
    # N number of path samples
    # NperLoop number of path samples done in one loop
    # T final time
    # r risk free interest rate
    # sig vol parameter
    # S0 starting underlying value
    # K strike value
    # exBoundType flag which denotes the parametrisation
    # type of the exercise boundary
    # exBoundParams parameters for the exercise boundary
    
    nf = M**l
    nc = M**(l-1)
    
    hf = T/nf
    hc = T/nc
    
    tf = 0
    tc = 0
    sums = np.zeros(4)
    
    # Loop until N paths have been computed in total
    for N1 in range(1, N+1, NperLoop):
        
        # at each step deal with N2 paths
        N2 = min(NperLoop, N-N1+1)
        
        Xf = np.zeros((N2, 4))
        
        # path variables X_t = log(S_t)
        Xf[:,0] = np.log(S0) * np.ones(N2)
        
        # Probability of not having crossed yet
        Xf[:,1] = np.ones(N2)
        
        # Payoff part 1 --> barrier hit during some interval
        Xf[:,2] = np.zeros(N2)
        Xc = Xf
        tf = 0
        tc = 0
        
        if l == 0:
            for n in range(1,nf+1):
                hf = T/nf**2 * (2*(nf-n)+1)
                dWf = np.sqrt(hf)*np.random.randn(N2)
                
                Xleft = Xf[:,0]
                Xright = Xleft + (r-sig**2/2)*hf + sig*dWf
                
                leftBarrier = exerciseBoundary(tf, T, K, exBoundType, exBoundParams)
                tf = tf + hf
                tfMid = tf - hf/2
                rightBarrier = exerciseBoundary(tf, T, K, exBoundType, exBoundParams)
                midBarrier = exerciseBoundary(tfMid,T,K,exBoundType,exBoundParams)
                
                if (leftBarrier != 0) and (rightBarrier != 0):
                    Prob1 = np.exp(-2 * max((Xleft - np.log(leftBarrier)).any(), 0) * max((Xright - np.log(rightBarrier)).any(),0) / hf / sig**2)
                    
                else:
                    Prob1 = np.exp(-2 * max((np.exp(Xleft)-leftBarrier).any(), 0) * max((np.exp(Xright)-rightBarrier).any(), 0) / hf / sig**2 / (np.exp(Xleft))**2)
                    
                # update payoff
                Xf[:,2] = Xf[:,2] + Xf[:,1]*Prob1 * max(K-midBarrier,0)*np.exp(-r*tfMid)
                
                # update crossing probability
                Xf[:,1] = Xf[:,1] * (1.0-Prob1)
                # update path variable
                Xf[:,0] = Xright
                
        else:
            for n in range(1,nc):
                dWc = np.zeros((N2,3))
                dt = np.zeros(2)
                
                for m in range(0,2):
                    hf = T/nf**2 * ( 2*(nf-2*(n-1)-m)+1)
                    dt[m] = hf
                    dWf = np.sqrt(hf)*np.random.randn(N2)
                    dWc[:,m] = dWf
                    Xleft = Xf[:,0]
                    
                    # advance path variable
                    Xright = Xleft + (r-sig**2/2)*hf + sig*dWf
                    
                    # evaluate barrier on left end point
                    leftBarrier = exerciseBoundary(tf,T,K,exBoundType,exBoundParams)
                    tf =tf+hf
                    tfMid = tf - hf/2
                    
                    # evaluate barrier on right end point 
                    rightBarrier = exerciseBoundary(tf,T,K,exBoundType,exBoundParams)
                    
                    # evaluate barrier at mid point
                    midBarrier = exerciseBoundary(tfMid,T,K,exBoundType,exBoundParams)
                    
                    # prob of hitting the barrier
                    if (leftBarrier !=0) and (rightBarrier !=0):
                    
                        Prob1=np.exp(-2.0 * max((Xleft-np.log(leftBarrier)).any(),0) * max((Xright-np.log(rightBarrier)).any(),0) /hf/sig**2)
                
                    else:
                        Prob1=np.exp(-2.0 * max((np.exp(Xleft)-leftBarrier).any(),0) * max((np.exp(Xright)-rightBarrier).any(),0) /hf /sig**2/(np.exp(Xleft))**2)
                        
                    
                
                    # update payoff
    
                    Xf[:,2] = Xf[:,2] + Xf[:,1]*Prob1 *max(K-midBarrier,0)*np.exp(-r*(tfMid))
    
                    # update crossing probability
    
                    Xf[:,1] = Xf[:,1]* (1.0-Prob1)
    
                    # update path value
                    Xf[:,0] = Xright
                    
                hc = dt[0] + dt[1]

                Xleft = Xc[:,0]

                # advance path variable

                Xright = Xleft + (r-sig**2/2)*hc + sig*(dWc[:,0]+dWc[:,1])

                lambdas = dt[0]/hc
                Xmid = Xleft + lambdas*(Xright-Xleft) + sig*(dWc[:,0] - lambdas*(dWc[:,0]+dWc[:,1]))

                # evaluate barrier at left end point
                leftBarrier = exerciseBoundary(tc,T,K,exBoundType,exBoundParams)

                tc =tc+hc

                tcMid = tc - hc/2

                # evaluate barrier at right end point
                rightBarrier = exerciseBoundary(tc,T,K,exBoundType,exBoundParams)

                # evaluate barrier at mid point
                midBarrier2 = exerciseBoundary(tcMid,T,K,exBoundType,exBoundParams)

                # prob of hitting the barrier
                if (leftBarrier !=0) and (rightBarrier !=0):
                
                    logLeftBarrier = np.log(leftBarrier)
                    logRightBarrier = np.log(rightBarrier)
                    logMidBarrier1 = logLeftBarrier + lambdas*(logRightBarrier-logLeftBarrier)
                    Prob11 = np.exp(-2*max((Xleft-logLeftBarrier).any(),0) * max((Xmid-logMidBarrier1).any(),0)     /dt[0]/sig**2)
                    Prob12 = np.exp(-2*max((Xmid-logMidBarrier1).any(),0) * max((Xright-logRightBarrier).any(),0) /dt[1]/sig**2)
            
                else:
                
                    midBarrier1 = leftBarrier + lambdas*(rightBarrier-leftBarrier)
                    Prob11 = np.exp(-2*max((np.exp(Xleft)-leftBarrier).any(),0) * max((np.exp(Xmid)-midBarrier1).any(),0) /dt[0]/sig**2)/(np.exp(Xleft))**2
                    Prob12 = np.exp(-2*max((np.exp(Xmid)-midBarrier1).any(),0)* max((np.exp(Xright)-rightBarrier).any(),0)/dt[1]/sig**2/(np.exp(Xleft))**2)
            
                
                Prob1 = (1.0-(1.0-Prob11)*(1.0-Prob12))
                # update payoff
                Xc[:,2] = Xc[:,2] + Xc[:,1]*Prob1*max(K-midBarrier2,0)*np.exp(-r*tcMid)
                # update crossing probability
                Xc[:,1] = Xc[:,1] * (1-Prob1)
                # update path variable
                Xc[:,0] = Xright
        
        Pf = Xf[:,2] + Xf[:,1]*max((K-np.exp(Xf[:,0])).any(),0)*np.exp(-r*T)

        if l == 0:
            Pc = 0
        else:
            Pc = Xc[:,2] + Xc[:,1]*max((K-np.exp(Xc[:,0])).any(),0)*np.exp(-r*T)
    sums[0] += sum(Pf-Pc)
    sums[1] += sum((Pf-Pc)**2)
    sums[2] += sum(Pf)
    sums[3] += sum(Pf**2)
    
    return sums

In [43]:
def MLMCAmOp(eps, L, N, NperLoop, T, r, sig, S0, K, M, exBoundType, exBoundParams):
    converged = 0
    while converged == 0:
        # Initial Variance Estimate
        L = L+1
        suml = np.zeros((3,L+1))
        sums = MLMCAmOpLevelL(L, N, NperLoop, T, r, sig, S0, K, M, exBoundType, exBoundParams)
        suml[0, L] = N
        suml[1, L] = sums[0]
        suml[2, L] = sums[1]
        
        # Optimal Sample Sizes
        Vl = suml[2,:]/suml[0,:] - (suml[1,:]/suml[0,:])**2
        Nl = np.ceil(2*np.sqrt(Vl/(2**np.arange(0,L+1))) * sum(np.sqrt(Vl*(2**np.arange(0,L+1)))) / eps**2)
        
        # Update Sample Sums
        for l in range(0,L+1):
            dNl = Nl[l]-suml[1,l]
            if dNl>0:
                sums = MLMCAmOpLevelL(l, dNl, NperLoop, T, r, sig, S0, K, exBoundType, exBoundParams)
                suml[0,l] += dNl
                suml[1,l] += sums[0]
                suml[2,l] += sums[1]
        
        # Test for Convergence
#         ranges = np.arange(-1, 0)
        if L>1 and 2**L >= 16:
            con = [2**-1, 2**0] * suml[1,L:L+1] / suml[0, L:L+1]
            converged = (max(abs(con)) < eps/np.sqrt(2))
            
    # Evaluate Multi-timestep Estimator
    P = sum(suml[1, :] / suml[0, :])
    Nl = suml[0,:]
    
    return [P, Nl]

In [ ]:
import numpy as np

p=[]
eps=np.array([0.005,0.01,0.02,0.05,0.1])
M = 2
T = 1
r = 0.05
sig = 0.2
L = -1
NperLoop=500
S0=1.0
K=100
N=100000
exBoundType = 3
exBoundParams = [0.8]

print(MLMCAmOp(0.1, L, N, NperLoop, T, r, sig, S0, K, M, exBoundType, exBoundParams))
# for e in eps:
#     [P, Nl] = MLMCAmOp(e, L, N, NperLoop, T, r, sig, S0, K, M, exBoundType, exBoundParams)
#     print(P)
# #     p += [np.sum(P[0,:]/Nl)]

# plt.plot(eps,p,label='MLMC')
# plt.fill_between(eps, p+eps, p-eps, alpha=.5)
# plt.xlabel('$\epsilon$')
# plt.ylabel('American Call Option Price')
# plt.title(f'American Call Option, $T={T}, S(0)={X0}, K={K}, r={r*100}\%, \sigma={sig}$');


In [7]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}